In [1]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO

In [2]:
url = "https://fbref.com/en/comps/9/Premier-League-Stats"
data = requests.get(url)

In [3]:
soup = BeautifulSoup(data.text)


In [4]:
stats_table = soup.select('table.stats_table')[0]

In [6]:
links = stats_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [7]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [8]:
data = requests.get(team_urls[0])
html_buffer = StringIO(data.text)
matches = pd.read_html(html_buffer, match="Scores & Fixtures")[0]
matches.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes
0,2023-08-06,16:00,Community Shield,FA Community Shield,Sun,Neutral,D,1 (1),1 (4),Arsenal,NaN,NaN,55,81145.0,Kyle Walker,4-2-3-1,Stuart Attwell,Match Report,Arsenal won on penalty kicks following normal ...
1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,1.9,0.3,65,21572.0,Kevin De Bruyne,4-2-3-1,Craig Pawson,Match Report,NaN
2,2023-08-16,22:00,Super Cup,UEFA Super Cup,Wed,Home,D,1 (5),1 (4),es Sevilla,NaN,NaN,74,NaN,Kyle Walker,4-2-3-1,François Letexier,Match Report,NaN
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,1.0,0.3,59,53419.0,Kyle Walker,4-2-3-1,Robert Jones,Match Report,NaN
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,3.5,0.7,79,31336.0,Kyle Walker,4-2-3-1,Jarred Gillett,Match Report,NaN


In [9]:
# el tag c9 trae los datos de la Premier League

soup = BeautifulSoup(html_buffer)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'c9/schedule/' in l]

In [16]:
#Fixture completo de las 38 fechas jugadas en la temporada 2023/2024

data = requests.get(f"https://fbref.com{links[0]}")
man_city_premier_stats = pd.read_html(data.text, match="Scores & Fixtures")[0]
man_city_premier_stats.head()
time.sleep(1)

C:\Windows\Temp\ipykernel_18416\3071462477.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  man_city_premier_stats = pd.read_html(data.text, match="Scores & Fixtures")[0]


In [17]:
man_city_premier_stats = man_city_premier_stats.drop(columns=['Match Report'])

In [21]:
man_city_premier_stats = man_city_premier_stats.drop(columns=['Notes'])

In [18]:
man_city_premier_stats.set_index('Round', inplace=True)

In [22]:
man_city_premier_stats.to_csv('matches.csv')

In [23]:
#Scrapeo Tabla de Tiros del Man.City
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'c9/shooting/' in l]

In [24]:
data = requests.get(f"https://fbref.com{links[0]}")
shooting = pd.read_html(data.text, match="Shooting")[0]
time.sleep(1)

C:\Windows\Temp\ipykernel_18416\1615510361.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


In [26]:
shooting.columns = shooting.columns.droplevel()

In [28]:
shooting.set_index('Round', inplace=True)

In [29]:
shooting.columns

Index(['Date', 'Time', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent', 'Gls',
       'Sh', 'SoT', 'SoT%', 'G/Sh', 'G/SoT', 'Dist', 'FK', 'PK', 'PKatt', 'xG',
       'npxG', 'npxG/Sh', 'G-xG', 'np:G-xG'],
      dtype='object')

In [53]:
columns_to_drop = ['Date', 'Time', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent','Match Report']
shooting.drop(columns= columns_to_drop, inplace=True)

In [54]:
shooting.head()

,Gls,Sh,SoT,SoT%,G/Sh,G/SoT,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG
Round,,,,,,,,,,,,,,,
Matchweek 1,3,17,8,47.1,0.18,0.38,13.9,0,0,0,1.9,1.9,0.12,1.1,1.1
Matchweek 2,1,14,4,28.6,0.07,0.25,17.9,0,0,0,1.0,1.0,0.07,0.0,0.0
Matchweek 3,2,29,9,31.0,0.07,0.22,17.3,2,0,1,3.5,2.8,0.10,-1.5,-0.8
Matchweek 4,5,6,4,66.7,0.67,1.00,14.8,0,1,1,2.2,1.4,0.25,2.8,2.6
Matchweek 5,3,29,13,44.8,0.10,0.23,16.4,1,0,0,3.6,3.6,0.13,-0.6,-0.6


In [30]:
#Scrapeo Tabla de Goles y Creación de Tiros del Man.City
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'c9/gca/' in l]


In [31]:
data = requests.get(f"https://fbref.com{links[0]}")
gca = pd.read_html(data.text, match="Goal and Shot Creation")[0]
time.sleep(1)

C:\Windows\Temp\ipykernel_18416\3695685396.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  gca = pd.read_html(data.text, match="Goal and Shot Creation")[0]


In [43]:
gca.columns = gca.columns.droplevel()
gca = gca.drop(columns=['Match Report'])


In [44]:
# Establecer "Round" como el índice
gca.set_index('Round', inplace=True)


In [47]:
# Nuevos nombres de las columnas
new_column_names = [
    'Date', 'Time', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent', 'SCA',
    'sca_PassLive', 'sca_PassDead', 'sca_TO', 'sca_Sh', 'sca_Fld', 'sca_Def', 
    'GCA', 'gca_PassLive', 'gca_PassDead', 'gca_TO', 'gca_Sh', 'gca_Fld', 'gca_Def'
]

# Asignar los nuevos nombres de las columnas utilizando la ubicación por índice
gca.columns = new_column_names

In [48]:
gca.columns

Index(['Date', 'Time', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent', 'SCA',
       'sca_PassLive', 'sca_PassDead', 'sca_TO', 'sca_Sh', 'sca_Fld',
       'sca_Def', 'GCA', 'gca_PassLive', 'gca_PassDead', 'gca_TO', 'gca_Sh',
       'gca_Fld', 'gca_Def'],
      dtype='object')

In [50]:
gca.drop(columns= columns_to_drop, inplace=True)

In [51]:
gca.head()

,SCA,sca_PassLive,sca_PassDead,sca_TO,sca_Sh,sca_Fld,sca_Def,GCA,gca_PassLive,gca_PassDead,gca_TO,gca_Sh,gca_Fld,gca_Def
Round,,,,,,,,,,,,,,
Matchweek 1,29,20,3,1,2,3,0,5,4,0,0,1,0,0
Matchweek 2,27,24,1,1,1,0,0,2,2,0,0,0,0,0
Matchweek 3,58,52,1,1,2,2,0,4,3,0,0,1,0,0
Matchweek 4,13,9,1,1,1,1,0,9,5,1,1,1,1,0
Matchweek 5,55,45,1,2,5,1,1,6,6,0,0,0,0,0


In [55]:
offensive_stats = pd.merge(shooting, gca, on='Round', how='inner')

In [60]:
offensive_stats.head()

,Gls,Sh,SoT,SoT%,G/Sh,G/SoT,Dist,FK,PK,PKatt,...,sca_Sh,sca_Fld,sca_Def,GCA,gca_PassLive,gca_PassDead,gca_TO,gca_Sh,gca_Fld,gca_Def
Round,,,,,,,,,,,,,,,,,,,,,
Matchweek 1,3,17,8,47.1,0.18,0.38,13.9,0,0,0,...,2,3,0,5,4,0,0,1,0,0
Matchweek 2,1,14,4,28.6,0.07,0.25,17.9,0,0,0,...,1,0,0,2,2,0,0,0,0,0
Matchweek 3,2,29,9,31.0,0.07,0.22,17.3,2,0,1,...,2,2,0,4,3,0,0,1,0,0
Matchweek 4,5,6,4,66.7,0.67,1.00,14.8,0,1,1,...,1,1,0,9,5,1,1,1,1,0
Matchweek 5,3,29,13,44.8,0.10,0.23,16.4,1,0,0,...,5,1,1,6,6,0,0,0,0,0


In [61]:
offensive_stats.to_csv("shooting_stats.csv")

In [62]:
#Scrapeo Tabla de Pases del Man.City
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'c9/passing/' in l]

In [63]:
data = requests.get(f"https://fbref.com{links[0]}")
passing = pd.read_html(data.text, match="Passing")[0]
time.sleep(1)


C:\Windows\Temp\ipykernel_18416\1354743369.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  passing = pd.read_html(data.text, match="Passing")[0]


In [66]:
# Renombrar las columnas especificas
passing_new_columns = []
for col in passing.columns:
    if col[1] in ['Cmp', 'Att', 'Cmp%']:
        passing_new_columns.append((col[0], f"{col[0]}_{col[1]}"))
    else:
        passing_new_columns.append(col)

In [69]:
passing.columns = passing.columns.from_tuples(passing_new_columns)

In [71]:
passing.columns = passing.columns.droplevel()

In [73]:
columns_to_drop = ['Date', 'Time', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent','Match Report']

In [74]:
passing.drop(columns= columns_to_drop, inplace=True)

In [75]:
passing.columns

Index(['Round', 'Total_Cmp', 'Total_Att', 'Total_Cmp%', 'TotDist', 'PrgDist',
       'Short_Cmp', 'Short_Att', 'Short_Cmp%', 'Medium_Cmp', 'Medium_Att',
       'Medium_Cmp%', 'Long_Cmp', 'Long_Att', 'Long_Cmp%', 'Ast', 'xAG', 'xA',
       'KP', '1/3', 'PPA', 'CrsPA', 'PrgP'],
      dtype='object')

In [77]:
# Establecer "Round" como el índice
passing.set_index('Round', inplace=True)

In [78]:
passing.head()

,Total_Cmp,Total_Att,Total_Cmp%,TotDist,PrgDist,Short_Cmp,Short_Att,Short_Cmp%,Medium_Cmp,Medium_Att,...,Long_Att,Long_Cmp%,Ast,xAG,xA,KP,1/3,PPA,CrsPA,PrgP
Round,,,,,,,,,,,,,,,,,,,,,
Matchweek 1,655,740,88.5,10790,3194,325,339,95.9,252,277,...,84,60.7,2,1.3,1.3,14,42,10,2,28
Matchweek 2,632,714,88.5,10155,2989,343,366,93.7,234,263,...,58,65.5,1,1.0,0.8,13,29,7,0,42
Matchweek 3,707,793,89.2,12243,3342,311,326,95.4,326,352,...,81,67.9,2,2.5,2.7,25,50,23,3,75
Matchweek 4,681,766,88.9,11763,3078,305,329,92.7,288,311,...,90,72.2,3,0.9,0.6,5,39,8,1,38
Matchweek 5,676,747,90.5,10979,2860,337,359,93.9,280,301,...,63,74.6,3,3.3,3.5,24,60,18,3,75


In [79]:
#Scrapeo Tabla de Tipos de Pases del Man.City
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'c9/passing_types/' in l]


In [80]:
data = requests.get(f"https://fbref.com{links[0]}")
pass_types = pd.read_html(data.text, match="Pass Types")[0]
time.sleep(1)

C:\Windows\Temp\ipykernel_18416\1095874312.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pass_types = pd.read_html(data.text, match="Pass Types")[0]


In [ ]:
pass_types.columns = pass_types.columns.droplevel()

In [110]:
columns_to_drop = ['Date', 'Time', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent','Match Report']

In [95]:
pass_types = pass_types.drop(columns=columns_to_drop)

In [104]:
pass_types = pass_types.drop(columns=['Match Report'])

In [105]:
pass_types.columns

Index(['Att', 'Live', 'Dead', 'FK', 'TB', 'Sw', 'Crs', 'TI', 'CK', 'In', 'Out',
       'Str', 'Cmp', 'Off', 'Blocks'],
      dtype='object')

In [97]:
# Establecer "Round" como el índice
pass_types.set_index('Round', inplace=True)

In [106]:
pass_types.head()

,Att,Live,Dead,FK,TB,Sw,Crs,TI,CK,In,Out,Str,Cmp,Off,Blocks
Round,,,,,,,,,,,,,,,
Matchweek 1,740,703,36,10,7,1,15,13,5,0,4,0,655,1,7
Matchweek 2,714,671,42,15,4,4,7,19,3,1,0,0,632,1,4
Matchweek 3,793,752,39,8,3,4,35,11,12,6,2,0,707,2,9
Matchweek 4,766,722,43,12,4,3,9,21,4,0,3,0,681,1,11
Matchweek 5,747,712,35,7,4,2,25,13,11,0,6,0,676,0,10


In [107]:
#Scrapeo Tabla de Posesión del Man.City
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'c9/possession/' in l]

In [108]:
data = requests.get(f"https://fbref.com{links[0]}")
possession = pd.read_html(data.text, match="Possession")[0]
time.sleep(1)

C:\Windows\Temp\ipykernel_18416\3595646561.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  possession = pd.read_html(data.text, match="Possession")[0]


In [111]:
possession.columns = possession.columns.droplevel()


In [112]:
possession = possession.drop(columns=columns_to_drop)

In [114]:
# Establecer "Round" como el índice
possession.set_index('Round', inplace=True)


In [113]:
possession.columns

Index(['Round', 'Poss', 'Touches', 'Def Pen', 'Def 3rd', 'Mid 3rd', 'Att 3rd',
       'Att Pen', 'Live', 'Att', 'Succ', 'Succ%', 'Tkld', 'Tkld%', 'Carries',
       'TotDist', 'PrgDist', 'PrgC', '1/3', 'CPA', 'Mis', 'Dis', 'Rec',
       'PrgR'],
      dtype='object')

In [115]:
possession.head()

,Poss,Touches,Def Pen,Def 3rd,Mid 3rd,Att 3rd,Att Pen,Live,Att,Succ,...,Carries,TotDist,PrgDist,PrgC,1/3,CPA,Mis,Dis,Rec,PrgR
Round,,,,,,,,,,,,,,,,,,,,,
Matchweek 1,65.0,835,75,267,412,158,23,835,10,3,...,558,2724,1240,12,13,3,9,9,632,28
Matchweek 2,59.0,788,65,212,411,174,21,788,19,14,...,434,2418,1223,25,16,9,11,5,627,42
Matchweek 3,79.0,893,40,153,440,304,53,892,17,9,...,676,3919,2285,42,40,10,13,5,700,73
Matchweek 4,68.0,838,64,222,483,138,20,837,17,12,...,524,2539,1472,18,10,3,9,7,673,38
Matchweek 5,68.0,837,49,148,382,311,49,837,15,7,...,678,3574,2229,53,27,15,9,7,670,74


In [116]:
passing_stats = pd.merge(passing, pass_types, on='Round', how='inner')

In [117]:
passing_stats = pd.merge(passing_stats, possession, on='Round', how='inner')

In [118]:
passing_stats.head()

,Total_Cmp,Total_Att,Total_Cmp%,TotDist_x,PrgDist_x,Short_Cmp,Short_Att,Short_Cmp%,Medium_Cmp,Medium_Att,...,Carries,TotDist_y,PrgDist_y,PrgC,1/3_y,CPA,Mis,Dis,Rec,PrgR
Round,,,,,,,,,,,,,,,,,,,,,
Matchweek 1,655,740,88.5,10790,3194,325,339,95.9,252,277,...,558,2724,1240,12,13,3,9,9,632,28
Matchweek 2,632,714,88.5,10155,2989,343,366,93.7,234,263,...,434,2418,1223,25,16,9,11,5,627,42
Matchweek 3,707,793,89.2,12243,3342,311,326,95.4,326,352,...,676,3919,2285,42,40,10,13,5,700,73
Matchweek 4,681,766,88.9,11763,3078,305,329,92.7,288,311,...,524,2539,1472,18,10,3,9,7,673,38
Matchweek 5,676,747,90.5,10979,2860,337,359,93.9,280,301,...,678,3574,2229,53,27,15,9,7,670,74


In [119]:
passing_stats.to_csv("passing_stats.csv")